In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import re
import string
import nltk
import nltk
nltk.download('wordnet')
import warnings
%matplotlib inline
warnings.filterwarnings("ignore")

In [ ]:
#from google.colab import files
#uploaded = files.upload()

In [ ]:
df = pd.read_csv("good_tweets_combined.csv")
df.head()

In [ ]:
df[df['target']==0].head()

In [ ]:
df[df['target']==1].head()

In [ ]:
#preprocessing


#url is not done yet

def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for word in r:
        input_txt = re.sub(word, "", input_txt)
    return input_txt

In [ ]:
#remove twitter handles (@user)
#vectorize means pass each row to the function and return
df['clean_tweet'] = np.vectorize(remove_pattern)(df['Tweet'], "@[\w]*")

In [ ]:
df.head()

In [ ]:
#remove special characters, numbers and punctuations
df['clean_tweet'] = df['clean_tweet'].str.replace("[^a-zA-Z#]", " ")
df.head()

In [ ]:
#remove short words
#if length is more than 3 then it will be added to string otherwise it will be ignored
df['clean_tweet'] = df['clean_tweet'].apply(lambda x: " ".join([w for w in x.split() if len(w)>3]))
df.head()

In [ ]:
#tokenization (individual word called tokens)
df['clean_tweet'] = df['clean_tweet'].apply(lambda x: x.split())

In [ ]:
#stem the words
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

df['clean_tweet'] = df['clean_tweet'].apply(lambda y :  [lemmatizer.lemmatize(w) for w in y])

In [ ]:
#combine all words into sentence
for i in range(len(df['clean_tweet'])):    
    df['clean_tweet'][i] = ' '.join(df['clean_tweet'][i])

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

In [ ]:
print(set(stopwords.words('english')))

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
df['clean_tweet'] = df['clean_tweet'].apply(lambda x : ' '.join([w for w in x.split() if not w in stop_words]))

In [ ]:
df.head()

In [ ]:
#visulization

In [ ]:
df.shape #number of (raws, columns)

In [ ]:
from tf.keras.preprocessing.text import one_hot
onehot_repr = [one_hot(words,5000) for words in df['clean_tweet']]

In [ ]:
from keras.preprocessing.sequence import pad_sequences
embedded_doc = pad_sequences(onehot_repr , padding = 'pre' , maxlen = 35)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense , Dropout , LSTM , Bidirectional , Embedding

model=Sequential()
model.add(Embedding(5000 , 50 ,input_length=35))
model.add(LSTM(50))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(2,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

In [ ]:


from sklearn.model_selection import train_test_split
X = np.array(embedded_doc)
Y = df['target']

#train_y = Y[:3300]
#test_y = Y[3300:]
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 40)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [ ]:
Y_train = np.asarray(Y_train).astype('float32').reshape((-1,1))
Y_test = np.asarray(Y_test).astype('float32').reshape((-1,1))

In [ ]:
from tensorflow.keras.utils import to_categorical
Y = to_categorical(Y)

In [ ]:
Y.shape

In [ ]:
history = model.fit(X, Y, validation_data=(X, Y), epochs=10, batch_size=64)

In [ ]:
plt.plot(history.history['loss'], 'r', label='Training loss')
plt.plot(history.history['val_loss'], 'g', label='Validation loss')
plt.title('Training VS Validation loss')
plt.xlabel('No. of Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['accuracy'], 'r', label='Training accuracy')
plt.plot(history.history['val_accuracy'], 'g', label='Validation accuracy') 
plt.title('Training Vs Validation Accuracy') 
plt.xlabel('No. of Epochs') 
plt.ylabel('Accuracy') 
plt.legend() 
plt.show()

In [ ]:
scores = model.evaluate(X, Y, verbose=0) 
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
scores = model.evaluate(X, Y, verbose=0) 
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
X_train.shape

In [ ]:
!pip install h5py

In [ ]:
# saving the model 
from keras.models import load_model

model.save('PlzGod.h5')

In [ ]:
import h5py

filename = "PlzGod.h5"

h5 = h5py.File(filename,'r')

#futures_data = h5['futures_data']  # VSTOXX futures data
#options_data = h5['options_data']  # VSTOXX call option data

h5.close()

In [ ]:
 li = []
 def chat():
     while True:
             inp = [input()]
             if inp[0] == 'quit':
                 break
             onehot_repr=[one_hot(words,5000)for words in inp]
             embedded_doc = pad_sequences(onehot_repr , padding = 'pre' , maxlen = 35)
             Xt = np.array(embedded_doc)
             res = np.argmax(model.predict(Xt) , axis=-1)
             #res = [item for sublist in res for item in sublist]
             print(res)

In [ ]:
chat()